# Задание 6. Классификация новостей

### Данные
Данные в [архиве](https://drive.google.com/file/d/15o7fdxTgndoy6K-e7g8g1M2-bOOwqZPl/view?usp=sharing). В нём два файла:
- `news_train.txt` тестовое множество
- `news_test.txt` тренировочное множество

С некоторых новостных сайтов были загружены тексты новостей за период  несколько лет, причем каждая новость принаделжит к какой-то рубрике: `science`, `style`, `culture`, `life`, `economics`, `business`, `travel`, `forces`, `media`, `sport`.

В каждой строке файла содержится метка рубрики, заголовок новостной статьи и сам текст статьи, например:

>    **sport**&nbsp;&lt;tab&gt;&nbsp;**Сборная Канады по хоккею разгромила чехов**&nbsp;&lt;tab&gt;&nbsp;**Сборная Канады по хоккею крупно об...**

## Задание 6.1 

Обработать данные, получив для каждого текста набор токенов
Обработать токены с помощью (один вариант из трех):

- pymorphy2
- русского [snowball стеммера](https://www.nltk.org/howto/stem.html)
- [SentencePiece](https://github.com/google/sentencepiece) или [Huggingface Tokenizers](https://github.com/huggingface/tokenizers)
    
    
## Задание 6.2

Обучить word embeddings (fastText, word2vec, gloVe) на тренировочных данных. Можно использовать [gensim](https://radimrehurek.com/gensim/models/word2vec.html) . Продемонстрировать семантические ассоциации. 

## Задание 6.3

Реализовать алгоритм классификации документа по категориям, посчитать точноть на тестовых данных, подобрать гиперпараметры. Метод векторизации выбрать произвольно - можно использовать $tf-idf$ с понижением размерности (см. scikit-learn), можно использовать обученные на предыдущем шаге векторные представления, можно использовать [предобученные модели](https://rusvectores.org/ru/models/). Имейте ввиду, что простое "усреднение" токенов в тексте скорее всего не даст положительных результатов. Нужно реализовать два алгоритмов из трех:
- SVM
- наивный байесовский классификатор
- логистическая регрессия
    

## Задание 6.4* 

Реализуйте классификацию с помощью нейросетевых моделей. Например [RuBERT](http://docs.deeppavlov.ai/en/master/features/models/bert.html) или [ELMo](https://rusvectores.org/ru/models/).

In [70]:
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\liucuiru991018\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [71]:
news_test=pd.read_table('news_test.txt',names=['category','title','text']).dropna()
news_train=pd.read_table('news_train.txt',names=['category','title','text']).dropna()
print(type(news_train))
news_test.head(10)

<class 'pandas.core.frame.DataFrame'>


,category,title,text
0,culture,Жительница Ямала победила в первом песенном ко...,Жительница Ямало-Ненецкого автономного округа ...
1,media,Почти половина Twitter-пользователей никогда н...,Около 44 процентов из всех зарегистрированных ...
2,media,"Билайн начал рекламу роуминга под песенку ""Тро...",В новой рекламной кампании мобильного оператор...
3,business,"Saipem потеряла 1,2 миллиарда евро из-за отмен...",Дочерняя структура итальянского нефтегазового ...
4,culture,Вин Дизель назвал «Форсаж 7» достойным «Оскара»,"Актер Вин Дизель заявил, что боевик «Форсаж 7»..."
5,culture,Майли Сайрус разденет фанатов и обольет их мол...,Американская певица Майли Сайрус появится обна...
6,life,Семейная пара выиграла миллион долларов по пут...,Семейная пара из штата Арканзас выиграла в одн...
7,science,Математики оправдали активность планктона,Ученые из Массачусетского технологического инс...
8,media,Файлохранилище RapidShare пригрозило сборщикам...,"Компания RapidShare, одно из крупнейших файлох..."
9,media,Google откажется от боковой панели в поисковике,Корпорация Google начала обновление интерфейса...


In [72]:
text1=news_train.text.values.tolist()
text2=news_test.text.values.tolist()
#print(text2[0])
print(type(text2))

<class 'list'>


In [73]:
Text=[]
for text in text2:
    clear=re.sub(r'[a-zA-Z0-9,—«».!;``:?`"''"&*%^)$#_(]','',text)
    #clear_text=re.sub(r'['',—«».!;:?`]','',clear)
    Text.append(word_tokenize(clear))
    
Text[0]
#print(len(Text))

['Жительница',
 'Ямало-Ненецкого',
 'автономного',
 'округа',
 'Елена',
 'Лаптандер',
 'победила',
 'в',
 'первом',
 'всероссийском',
 'песенном',
 'конкурсе',
 'Новая',
 'звезда',
 'сообщили',
 'Лентеру',
 'организаторы',
 'события',
 'В',
 'качестве',
 'награды',
 'ей',
 'досталась',
 'статуэтка',
 'в',
 'форме',
 'звезды',
 'и',
 'денежный',
 'приз',
 'один',
 'миллион',
 'рублей',
 'По',
 'словам',
 'финалистки',
 'вознаграждение',
 'она',
 'планирует',
 'передать',
 'в',
 'благотворительный',
 'фонд',
 'Подари',
 'жизньТри',
 'дополнительных',
 'приза',
 'достались',
 'Руслану',
 'Ивакину',
 'из',
 'Хакасии',
 'фолк-группе',
 'Ярилов',
 'Зной',
 'из',
 'Воронежской',
 'области',
 'и',
 'Александру',
 'Куулару',
 'из',
 'Тывы',
 'Призеров',
 'с',
 'помощью',
 'смс-голосования',
 'выбирали',
 'зрители',
 'телеканала',
 'Звезда',
 'который',
 'транслировал',
 'конкурс',
 'Всего',
 'на',
 'финальном',
 'гала-концерте',
 'выступили',
 'участниковВедущими',
 'шоу',
 'стали',
 'актеры',


In [74]:
from nltk.stem.snowball import SnowballStemmer 
stemmer = SnowballStemmer("russian") 

tk=[]
for ttext in Text:
    I = [stemmer.stem(word) for word in ttext]
    tk.append(I)
    
tk[0]

['жительниц',
 'ямало-ненецк',
 'автономн',
 'округ',
 'ел',
 'лаптандер',
 'побед',
 'в',
 'перв',
 'всероссийск',
 'песен',
 'конкурс',
 'нов',
 'звезд',
 'сообщ',
 'лентер',
 'организатор',
 'событ',
 'в',
 'качеств',
 'наград',
 'е',
 'доста',
 'статуэтк',
 'в',
 'форм',
 'звезд',
 'и',
 'денежн',
 'приз',
 'один',
 'миллион',
 'рубл',
 'по',
 'слов',
 'финалистк',
 'вознагражден',
 'он',
 'планир',
 'переда',
 'в',
 'благотворительн',
 'фонд',
 'подар',
 'жизньтр',
 'дополнительн',
 'приз',
 'доста',
 'руслан',
 'ивакин',
 'из',
 'хакас',
 'фолк-групп',
 'ярил',
 'зно',
 'из',
 'воронежск',
 'област',
 'и',
 'александр',
 'куулар',
 'из',
 'тыв',
 'призер',
 'с',
 'помощ',
 'смс-голосован',
 'выбира',
 'зрител',
 'телекана',
 'звезд',
 'котор',
 'транслирова',
 'конкурс',
 'всег',
 'на',
 'финальн',
 'гала-концерт',
 'выступ',
 'участниковведущ',
 'шо',
 'стал',
 'актер',
 'серг',
 'безрук',
 'и',
 'аврор',
 'на',
 'концерт',
 'прозвуча',
 'популярн',
 'песн',
 'воен',
 'лет',
 'ж

In [75]:
from gensim.models import FastText
model_ted1 = FastText(Text, window=5, min_count=5,sg=1)

model_ted1.wv.most_similar('Новая')

[('Самая', 0.9482162594795227),
 ('Русская', 0.9365611672401428),
 ('Дубая', 0.93421471118927),
 ('Точная', 0.9324676990509033),
 ('Морская', 0.9298667311668396),
 ('Единая', 0.9257813692092896),
 ('Комсомольская', 0.9236453771591187),
 ('Поклонская', 0.9236075282096863),
 ('шведская', 0.9234118461608887),
 ('пивоваренная', 0.917478084564209)]

In [76]:
from gensim.models import word2vec
model_ted2 = word2vec.Word2Vec(sentences=Text, window=5, workers=4,min_count=5,sg=0)

model_ted2.wv.most_similar('форма')

[('Эксперты', 0.989836573600769),
 ('показало', 0.9879303574562073),
 ('Агентство', 0.9873213171958923),
 ('Эстонии', 0.9872106313705444),
 ('подразделения', 0.9870633482933044),
 ('легких', 0.9867696762084961),
 ('комментариев', 0.9867680072784424),
 ('корпорации', 0.9866123199462891),
 ('социальную', 0.9863418340682983),
 ('кампания', 0.9862088561058044)]